In [1]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np 

# Experimenting with Taguchi Method

In [2]:
class Variable: 
    _name = ""
    _values = []
    def __init__(self, name:str, values: list): 
        self._name = name
        self._values = values
    def __str__(self): 
        return f"Variable(name = '{self._name}', values = {self._values})"

    @property
    def values(self) -> list: 
        return self._values
    
    @property
    def name(self) -> str: 
        return self._name
        
    @property
    def n_values(self) -> int: 
        return len(self._values)

In [3]:
v = Variable('A', ['L', 'H'])
print(v)

Variable(name = 'A', values = ['L', 'H'])


## Taguchi Array  
This algorithm provides the Orthogonal(Taguchi) Array with the inputs: 
* Q (the number of the levels) 
* N (the number of the factors).
* The output is an M*N array where M = Q^J
*  the rows of the Taguchi table and J meets the equation N= (Q^(J-1) - 1)/(Q-1);
*  J = log(N * (Q-1) + 1) / log(Q)

https://www.mathworks.com/matlabcentral/fileexchange/71628-taguchiarray

https://ieeexplore.ieee.org/document/910464/references#references

Reference: Leung, Y.-W.; Yuping Wang, "An orthogonal genetic algorithm
with quantization % for global numerical optimization," Evolutionary
Computation, IEEE Transactions on , vol.5, % no.1, pp.41,53, Feb 2001.

In [4]:
class TaguchiOpt: 
    _n_exp = 0
    _variables: list[Variable] = []
    _Q = 0
    _N = 0
    _J = 0 
    _M = 0 
    _OA = None
    _OA_values = None

    @classmethod
    def from_dict(cls, variables: dict): 
        _vars = []
        for k, v in variables.items(): 
            _vars.append(Variable(k, v))
        return cls(_vars)    
        
    def __init__(self, variables: list[Variable]): 
        self._variables = variables
        self._N = len(variables)
        self.build_OA()

    def build_OA(self) -> int:
        m_v = None 
        for v in self._variables: 
            if not m_v or v.n_values > m_v.n_values: 
                m_v = v 
        self._Q = m_v.n_values
        self._find_J()
        self._M = self._Q ** self._J
        self._OA = np.full((self._M, self._N), 2, dtype=int)
        self._OA_values = np.full((self._M, self._N), None)
        self._fill_OA()
        return self

    def _fill_OA(self):
        ## For Basic Columns
        for k in range(1, self._J+1): 
            j = int ( ((self._Q**(k-1)) - 1 ) \
                     / (self._Q - 1) ) + 1
            max_i = int(self._Q ** self._J)
            for i in range (1, max_i + 1): 
                den = (self._Q ** (self._J - k))
                # Fixing i,j for pyhton zero based array
                self._OA[i-1, j-1] = int((i - 1) / den) % self._Q #if den > 0 else None

        # For Non-Basic Columns
        for k in range (2, self._J+1): 
            j = int ( ((self._Q ** (k-1)) - 1 ) \
                     / (self._Q - 1) ) + 1
            for s in range (1, j): 
                for t in range (1, self._Q):
                    a_s = self._OA[:,s-1]
                    a_j = self._OA[:,j-1]
                    a_jj = int((j+(s-1)*(self._Q-1)+t)) - 1 
                    if a_jj < self._N:
                        self._OA[:,a_jj] = np.mod(a_s *t  + a_j, self._Q)

        self._OA = self._OA[:,0:self._N]

        for i in range (0, self._M): 
            for j in range(0, self._N): 
                v = self._variables[j]
                vi  = self._OA[i,j]
                try: 
                    value = v.values[vi]
                    self._OA_values[i, j] = value
                except: 
                    print(f"Error at {v}, {vi}, ({i,j})")
    
    def _calc_N_for_QJ(self): 
        return int( (self._Q**(self._J) - 1)/(self._Q-1) )
        
    def _find_J(self): 
        self._J = int(np.log(self._N * (self._Q-1) + 1) / np.log(self._Q))
        n = self._calc_N_for_QJ()
        if n > self._N: 
            self._J -= 1
        elif n < self._N: 
            self._J += 1
        self.N = self._calc_N_for_QJ() 
        
    @property
    def OA(self): 
        return self._OA_values

    def get_params(self, n=0): 
        return { self._variables[i].name : self._OA_values[n][i] for i in range(self._N)}
                
    def __str__(self): 
        return f"(Q = {self._Q}, N  = {self._N}, J = {self._J}, M = {self._M}, N = {self._N})"

variables = [
    Variable('A', ['N', 'L', 'H']), 
    Variable('B', ['N', 'L', 'H']), 
    Variable('C', ['N', 'L']),
]


OA = TaguchiOpt(variables)
print(OA)
print (OA.OA)

OA.get_params(2)

Error at Variable(name = 'C', values = ['N', 'L']), 2, ((2, 2))
Error at Variable(name = 'C', values = ['N', 'L']), 2, ((4, 2))
Error at Variable(name = 'C', values = ['N', 'L']), 2, ((6, 2))
(Q = 3, N  = 3, J = 2, M = 9, N = 3)
[['N' 'N' 'N']
 ['N' 'L' 'L']
 ['N' 'H' None]
 ['L' 'N' 'L']
 ['L' 'L' None]
 ['L' 'H' 'N']
 ['H' 'N' None]
 ['H' 'L' 'N']
 ['H' 'H' 'L']]


{'A': 'N', 'B': 'H', 'C': None}

In [5]:
variables = [
    Variable('A', [ 'L', 'H']), 
    Variable('B', [ 'L', 'H']), 
    Variable('C', [ 'L', 'H'])]
OA = TaguchiOpt(variables)
print(OA)
print (OA.OA)

(Q = 2, N  = 3, J = 2, M = 4, N = 3)
[['L' 'L' 'L']
 ['L' 'H' 'H']
 ['H' 'L' 'H']
 ['H' 'H' 'L']]


In [6]:
param_grid = {'max_depth': [5, 10, 20],
               'min_samples_split': [5, 10, 20], 
             'max_features' : ['sqrt', 'log2', None]}

In [7]:
OA = TaguchiOpt.from_dict(param_grid)
print(OA)
print (OA.OA)

(Q = 3, N  = 3, J = 2, M = 9, N = 3)
[[5 5 'sqrt']
 [5 10 'log2']
 [5 20 None]
 [10 5 'log2']
 [10 10 None]
 [10 20 'sqrt']
 [20 5 None]
 [20 10 'sqrt']
 [20 20 'log2']]


In [8]:
base_estimator = RandomForestClassifier(random_state=42)
X, y = make_classification(n_samples=5_000, n_features=100, n_informative=30, random_state=42)

In [9]:
%%time 
sh = GridSearchCV(base_estimator, param_grid, cv=5, verbose=2)
sh_fit = sh.fit(X, y)
sh_fit.best_params_

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END max_depth=5, max_features=sqrt, min_samples_split=5; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_split=5; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_split=5; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_split=5; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_split=5; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_split=10; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_split=10; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_split=10; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_split=10; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_split=10; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_split=20; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, 

{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 10}

In [10]:
len(sh.cv_results_['params'])

27

In [11]:
%%time
results = {
    'params': [], 
    'split0_test_score': [], 
    'split1_test_score': [],
    'split2_test_score': [],
    'split3_test_score': [],
    'split4_test_score': [],
    'mean_test_score': [], 
    'std_test_score': [],
    'max_depth': [],
    'min_samples_split': [], 
    'max_features' : []
}

best_model_params = None
best_model_score = 0
for e in range(len(OA.OA)): 
    params = OA.get_params(e)
    print(params)
    estimator = RandomForestClassifier(random_state=42, **params)
    split_scores = cross_val_score(estimator, X, y, cv=5)
    results['params'].append(params)
    for k, v in params.items(): 
        results[k].append(v)
    for i in range(5): 
        results[f'split{i}_test_score'].append(split_scores[i])
    mean_score  = np.mean(split_scores)
    results['mean_test_score'].append(mean_score) 
    if mean_score >= best_model_score: 
        best_model_params = params
        best_model_score = mean_score
        
    results['std_test_score'].append(np.std(split_scores))
    print()

{'max_depth': 5, 'min_samples_split': 5, 'max_features': 'sqrt'}

{'max_depth': 5, 'min_samples_split': 10, 'max_features': 'log2'}

{'max_depth': 5, 'min_samples_split': 20, 'max_features': None}

{'max_depth': 10, 'min_samples_split': 5, 'max_features': 'log2'}

{'max_depth': 10, 'min_samples_split': 10, 'max_features': None}

{'max_depth': 10, 'min_samples_split': 20, 'max_features': 'sqrt'}

{'max_depth': 20, 'min_samples_split': 5, 'max_features': None}

{'max_depth': 20, 'min_samples_split': 10, 'max_features': 'sqrt'}

{'max_depth': 20, 'min_samples_split': 20, 'max_features': 'log2'}

CPU times: total: 7min 21s
Wall time: 8min 43s


In [12]:
sh_fit.best_params_

{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 10}

In [13]:
sh_fit.best_score_

0.9019999999999999

In [14]:
df_results = pd.DataFrame(sh_fit.cv_results_)
cols = ['params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score']
df_results[cols].sort_values('mean_test_score', ascending=False).head(20)

,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score
19,"{'max_depth': 20, 'max_features': 'sqrt', 'min...",0.897,0.896,0.911,0.911,0.895,0.9020,0.007376
20,"{'max_depth': 20, 'max_features': 'sqrt', 'min...",0.897,0.891,0.906,0.904,0.899,0.8994,0.005314
18,"{'max_depth': 20, 'max_features': 'sqrt', 'min...",0.886,0.894,0.907,0.908,0.898,0.8986,0.008237
11,"{'max_depth': 10, 'max_features': 'sqrt', 'min...",0.883,0.888,0.900,0.901,0.892,0.8928,0.006911
10,"{'max_depth': 10, 'max_features': 'sqrt', 'min...",0.902,0.895,0.889,0.899,0.872,0.8914,0.010632
9,"{'max_depth': 10, 'max_features': 'sqrt', 'min...",0.903,0.888,0.896,0.893,0.871,0.8902,0.010759
22,"{'max_depth': 20, 'max_features': 'log2', 'min...",0.885,0.887,0.893,0.887,0.879,0.8862,0.004490
23,"{'max_depth': 20, 'max_features': 'log2', 'min...",0.878,0.888,0.889,0.899,0.875,0.8858,0.008565
12,"{'max_depth': 10, 'max_features': 'log2', 'min...",0.882,0.879,0.878,0.899,0.878,0.8832,0.008035
21,"{'max_depth': 20, 'max_features': 'log2', 'min...",0.886,0.882,0.890,0.893,0.864,0.8830,0.010198


In [15]:
best_model_params

{'max_depth': 20, 'min_samples_split': 10, 'max_features': 'sqrt'}

In [16]:
best_model_score

0.9019999999999999

In [17]:
pd.set_option('max_colwidth', 1001)

df_results_oa = pd.DataFrame(results)
df_results_oa.sort_values('mean_test_score', ascending=False).head(20)

,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,max_depth,min_samples_split,max_features
7,"{'max_depth': 20, 'min_samples_split': 10, 'max_features': 'sqrt'}",0.897,0.896,0.911,0.911,0.895,0.9020,0.007376,20,10,sqrt
5,"{'max_depth': 10, 'min_samples_split': 20, 'max_features': 'sqrt'}",0.883,0.888,0.900,0.901,0.892,0.8928,0.006911,10,20,sqrt
8,"{'max_depth': 20, 'min_samples_split': 20, 'max_features': 'log2'}",0.878,0.888,0.889,0.899,0.875,0.8858,0.008565,20,20,log2
3,"{'max_depth': 10, 'min_samples_split': 5, 'max_features': 'log2'}",0.882,0.879,0.878,0.899,0.878,0.8832,0.008035,10,5,log2
6,"{'max_depth': 20, 'min_samples_split': 5, 'max_features': None}",0.861,0.882,0.860,0.882,0.859,0.8688,0.010796,20,5,None
4,"{'max_depth': 10, 'min_samples_split': 10, 'max_features': None}",0.857,0.866,0.855,0.874,0.854,0.8612,0.007679,10,10,None
0,"{'max_depth': 5, 'min_samples_split': 5, 'max_features': 'sqrt'}",0.846,0.844,0.854,0.850,0.834,0.8456,0.006741,5,5,sqrt
1,"{'max_depth': 5, 'min_samples_split': 10, 'max_features': 'log2'}",0.835,0.821,0.843,0.847,0.832,0.8356,0.009069,5,10,log2
2,"{'max_depth': 5, 'min_samples_split': 20, 'max_features': None}",0.809,0.816,0.790,0.812,0.808,0.8070,0.008944,5,20,None


In [18]:
for k in params.keys(): 
    print(df_results_oa[[k, 'mean_test_score']].fillna('None').groupby(k).sum().reset_index().sort_values('mean_test_score', ascending=False))

   max_depth  mean_test_score
2         20           2.6566
1         10           2.6372
0          5           2.4882
   min_samples_split  mean_test_score
1                 10           2.5988
0                  5           2.5976
2                 20           2.5856
  max_features  mean_test_score
2         sqrt           2.6404
1         log2           2.6046
0         None           2.5370


In [19]:
param_grid2 = {'max_depth': [10, 20],
               'min_samples_split': [10], 
             'max_features' : ['sqrt']}

In [20]:
%%time 
sh = GridSearchCV(base_estimator, param_grid2, cv=5, verbose=2)
sh_fit = sh.fit(X, y)
sh_fit.best_params_

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_split=10; total time=   3.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_split=10; total time=   3.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_split=10; total time=   3.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_split=10; total time=   3.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_split=10; total time=   3.2s
[CV] END max_depth=20, max_features=sqrt, min_samples_split=10; total time=   4.0s
[CV] END max_depth=20, max_features=sqrt, min_samples_split=10; total time=   3.9s
[CV] END max_depth=20, max_features=sqrt, min_samples_split=10; total time=   3.9s
[CV] END max_depth=20, max_features=sqrt, min_samples_split=10; total time=   4.0s
[CV] END max_depth=20, max_features=sqrt, min_samples_split=10; total time=   3.9s
CPU times: total: 32.5 s
Wall time: 42.1 s


{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 10}

In [21]:
sh_fit.best_score_

0.9019999999999999